In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from model import Lenet as Lenet

## Data Processing
The data has been arranged in train and test directories. train.npy and test.npy contain the input images for the test and train sets. Likewise, train_cat.npy and test_cat.npy contain the corresponding labels.
We first load the data using `np.load()` and check out the shapes of numpy arrays using `np.shape`. 

In [2]:
train_data = np.load('norb/train.npy')
train_labels = np.load('norb/train_cat.npy')

test_data = np.load('norb/test.npy')
test_labels = np.load('norb/test_cat.npy')

# Let's print the shapes of these numpy arrays
print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)


# Let's also check the data type of these variables
print(type(train_data))
for i in range(10):
    print(train_labels[i])

(29160, 1, 108, 108) (29160,)
(29160, 1, 108, 108) (29160,)
<class 'numpy.ndarray'>
0
1
2
3
4
5
0
1
2
3


Notice that the test data is quite large in size. We'll deal with a small subset of the test set for validation and use the rest for testing later. For this, we'll have to slice the test set in its first dimension. 

In [3]:
# Choosing a subset (first 1000) of the test set for validation purposes.
test_data = test_data[:1000]
test_labels = test_labels[:1000]

# Let's verify
print(test_data.shape, test_labels.shape)

(1000, 1, 108, 108) (1000,)


### Converting our data from numpy arrays to PyTorch tensors.
So far, we've been working with numpy arrays. For performing further operations, like forward prop, accessing cuda, we should convert the numpy arrays into PyTorch tensors. It's simple: use `torch.from_numpy()` for this. Typecasting can be accomplished by simply calling the corresponding functions: `x.float()` or `x.long()`.

In [4]:
# Converting to PyTorch tensors
train_data = torch.from_numpy(train_data).float()
test_data = torch.from_numpy(test_data).float()

# If we're planning to use cross entropy loss, the data type of the
# targets needs to be 'long'.
train_labels = torch.from_numpy(train_labels).long()
test_labels = torch.from_numpy(test_labels).long()

print(type(train_data))

<class 'torch.Tensor'>


In [5]:
# When dealing with PyTorch tensors, it is recommended to use x.size()
# instead of x.shape to find the shape/size of the tensor
print(train_data.size())
print(train_data.size(0))

torch.Size([29160, 1, 108, 108])
29160


### Converting into Cuda tensors
Since we're going to use GPUs, the variables first need to be converted to cuda-type. For doing this, use `x = x.cuda()`. This, in effect, loads the tensors into your GPUs memory. This operation should be used very judiciously because if mishandled the data transfer itself could introduce major time delays. More on this later.

In [6]:
# Convert the data and labels into cuda Variables now: x = x.cuda()
train_data = train_data.cuda()
test_data = test_data.cuda()

train_labels = train_labels.cuda()
test_labels = test_labels.cuda()

# Let's do a sanity check
print(train_data.type())

torch.cuda.FloatTensor


Did you notice the slight execution delay in this operation? Yes, that's the time it took to transfer the data into GPUs memory. Also, notice that the tensor type now is `torch.cuda.FloatTensor`. To further verify that the data is actually physically existing in the GPUs memory, go to your terminal and run `$ nvidia-smi`. You should be able to see a python process listed using approx. 2GB of GPU memory. We're inching closer towards training our network.

__Note__: Converting the entire data into cuda variable is NOT a good practice.
We're still able to do it here because our data is small and can fit in
the GPU memory. When working with larger datasets (will see tomorrow) and,
bigger networks, it is strongly advised to convert only the minibatches into cuda just
before they're fed to the network.

### Introducing `torch.autograd.Variable`
So far, we've been dealing with PyTorch tensors very plainly. However, for them to be usable for deep learning operations, we also need to keep track of things like gradients of a tensor, if they are needed, for automatic gradient propagation. For this, we convert our tensors into objects of `torch.autograd.Variable` class. As we'll see, doing this brings our tensors to 'life', ready to handle the excruciatingly painful optimizations and backpropagation!!!

In [7]:
# Convert a tensor to a Variable object by simply asking it to track the gradients
train_data.requires_grad_(True)
test_data.requires_grad_(True)

# The targets/labels do not require gradients
train_labels.requires_grad_(False)
test_labels.requires_grad_(False)
print(train_labels)
print(train_labels.requires_grad)

tensor([ 0,  1,  2,  ...,  3,  4,  5], device='cuda:0')
False


### Creating the Network and setting up the Loss Function
We have created a sample network architecture for you in the file model.py. Check out its `create_fcn()` function. For initializing the losses, one may choose from a large variety of [losses available](https://pytorch.org/docs/stable/nn.html#loss-functions).

In [8]:
# create_fcn function is written in model.py.
model = Lenet()
# Initialise a loss function.
# eg. if we wanted an MSE Loss: loss_fn = nn.MSELoss()
# Please search the PyTorch doc for cross-entropy loss function
# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.CrossEntropyLoss()

Now, convert the model and the loss funtion into cuda types too. This is similar to what we did with the tensors.

In [9]:
model.cuda()
loss_fn.cuda()

CrossEntropyLoss()

### Introducing Optimizer
An optimizer is the basic engine that performs gradient descent with all its variants and hyperparameters. We need this module to be care-free about weight updates backpropagation. PyTorch provides a wide range of optimizers buil-in. Check out [this link](https://pytorch.org/docs/stable/optim.html) to explore them. Assuming we're using Adam optimizer, we can initialize it by calling `torch.optim.Adam()`. Note that while initializing, it needs to know all the network's parameters (weights and biases). This can be provided by using `model.parameters()`.

In [10]:
learning_rate = 0.0001
# Initializing the optimizer with hyperparameters.
# Please play with SGD, RMSProp, Adagrad, etc.
# Note that different optimizers may require differen hyperparameter values
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
# Initializing some training parameters
batch_size = 324

# number of batches in one epoch
n_batch = train_data.shape[0] // batch_size 
accuracy = 0.0
n_epoch = 2

### Entering the Training Loop
We'll now enter the training loop and do the following:
* Create a minibatch of size `batch_size` from the train data
* Forward propagate the minibatch through the network
* Compute the loss using the lost function defined previously
* Backpropagate the loss through the network (thanks to `torch.autograd`)
* Update the weights of the model using the `optimizer`
* Finally, compute the performance statistics

In [14]:
print(train_data.size())
for t in range(n_epoch):
     for m in range(n_batch):
         inp = train_data[m * batch_size: (m+1) * batch_size]
         tar = train_labels[m * batch_size: (m+1) * batch_size ]
 
         # Add random perturbations in this functions. Define
         # this function if you wish to use it.
         # inp = add_noise(inp)
 
         # Compute the network's output: Forward Prop
         pred = model(inp)
 
         # Compute the network's loss
         loss = loss_fn(pred, tar)
 
         # Zero the gradients of all the network's parameters
         optimizer.zero_grad()
 
         # Computer the network's gradients: Backward Prop
         loss.backward()
 
         # Update the network's parameters based on the computed
         # gradients
         optimizer.step()
 
         print(t, m, loss.item(), accuracy)
 
     # Validation after every 2nd epoch
     if t % 2 == 0:
         # Forward pass
         output = model(test_data)
 
         # get the index of the max log-probability
         pred = output.data.max(1)[1]
 
         correct = pred.eq(test_labels).sum()
         accuracy = correct.item() / 1000
         print("\n*****************************************\n")
         print(accuracy)
         print("\n*****************************************\n")


torch.Size([29160, 1, 108, 108])
0 0 0.4715828001499176 0.0
0 1 0.43570002913475037 0.0
0 2 0.4303221106529236 0.0
0 3 0.5127412676811218 0.0
0 4 0.5315393805503845 0.0
0 5 0.45340511202812195 0.0
0 6 0.42806583642959595 0.0
0 7 0.434641569852829 0.0
0 8 0.48168861865997314 0.0
0 9 0.47047993540763855 0.0
0 10 0.4525861442089081 0.0
0 11 0.4219578504562378 0.0
0 12 0.4365914463996887 0.0
0 13 0.43112102150917053 0.0
0 14 0.3594200909137726 0.0
0 15 0.4594959318637848 0.0
0 16 0.4714396297931671 0.0
0 17 0.43740707635879517 0.0
0 18 0.42703863978385925 0.0
0 19 0.400892972946167 0.0
0 20 0.5120314359664917 0.0
0 21 0.4627264142036438 0.0
0 22 0.5162467360496521 0.0
0 23 0.48216646909713745 0.0
0 24 0.4687061905860901 0.0
0 25 0.4201838970184326 0.0
0 26 0.4723787307739258 0.0
0 27 0.4708762466907501 0.0
0 28 0.44528627395629883 0.0
0 29 0.3720390796661377 0.0
0 30 0.5000483393669128 0.0
0 31 0.4863823652267456 0.0
0 32 0.36215898394584656 0.0
0 33 0.49830862879753113 0.0
0 34 0.47018432

That's our introduction to neural networks using PyTorch. Tomorrow, we'll try solving a more challenging problem with bigger dataset and more complicated network in a more principled manner! Hope to see you all tomorrow!